In [26]:
pip install python-docx

Note: you may need to restart the kernel to use updated packages.


In [27]:
ARTICLE_PATTERN = re.compile(r'^제\s?\d+조(?:의\d+)?\(')

ONLY_ARTICLE_NO = re.compile(r'^제\s?\d+조(?:의\d+)?$')

In [28]:
# 조문 파싱 코드
import os
from docx import Document
import re

ARTICLE_PATTERN = re.compile(r'^제\s?\d+조(?:의\d+)?\(')
ONLY_ARTICLE_NO = re.compile(r'^제\s?\d+조(?:의\d+)?$')

def parse_articles(docx_path: str):
    doc = Document(docx_path)

    articles = []
    current_article = None
    buffer = []

    for p in doc.paragraphs:
        line = p.text.strip()
        if not line:
            continue

        # ❌ 조문 번호만 있는 줄 제거
        if ONLY_ARTICLE_NO.match(line):
            continue

        # ✅ 조문 시작
        if ARTICLE_PATTERN.match(line):
            if current_article:
                articles.append({
                    "article": current_article,
                    "text": "\n".join(buffer)
                })

            current_article = ARTICLE_PATTERN.match(line).group()[:-1]
            buffer = [line]
        else:
            if current_article:
                buffer.append(line)

    if current_article and buffer:
        articles.append({
            "article": current_article,
            "text": "\n".join(buffer)
        })

    return articles

In [29]:
law_data = parse_law_folder("data/raw/law/")



📄 parsing: 민법(법률)(제20432호)(20260101)-간략.docx
📄 parsing: 상가건물 임대차보호법(법률)(제21065호)(20260102).docx
📄 parsing: 주택임대차보호법(법률)(제21065호)(20260102).docx


In [30]:
for a in law_data[fname][:5]:
    print(a["article"])
    print(a["text"][:100])
    print("------")

제1조
제1조(목적) 이 법은 주거용 건물의 임대차(賃貸借)에 관하여 「민법」에 대한 특례를 규정함으로써 국민 주거생활의 안정을 보장함을 목적으로 한다.
[전문개정 2008. 3. 21.
------
제2조
제2조(적용 범위) 이 법은 주거용 건물(이하 “주택”이라 한다)의 전부 또는 일부의 임대차에 관하여 적용한다. 그 임차주택(賃借住宅)의 일부가 주거 외의 목적으로 사용되는 경우에
------
제3조
제3조(대항력 등) ① 임대차는 그 등기(登記)가 없는 경우에도 임차인(賃借人)이 주택의 인도(引渡)와 주민등록을 마친 때에는 그 다음 날부터 제삼자에 대하여 효력이 생긴다. 이 
------
제3조의2
제3조의2(보증금의 회수) ① 임차인(제3조제2항 및 제3항의 법인을 포함한다. 이하 같다)이 임차주택에 대하여 보증금반환청구소송의 확정판결이나 그 밖에 이에 준하는 집행권원(執行
------
제3조의3
제3조의3(임차권등기명령) ① 임대차가 끝난 후 보증금이 반환되지 아니한 경우 임차인은 임차주택의 소재지를 관할하는 지방법원ㆍ지방법원지원 또는 시ㆍ군 법원에 임차권등기명령을 신청할
------
